In [ ]:
# --- 1. Installation (Run this cell once) ---
# !pip install aiograpi nest_asyncio

# --- 2. Imports and Setup ---
import asyncio
import json
import os
import python-dotenv  # To load environment variables from a .env file
# Load environment variables from .env file if it exists
dotenv_path = os.path.join(os.path.dirname(__file__), '.env')
if os.path.exists(dotenv_path):
    print(f"Loading environment variables from {dotenv_path}...")
    from dotenv import load_dotenv
    load_dotenv(dotenv_path)
else:
    print(f"No .env file found at {dotenv_path}. Skipping loading environment variables.")
from getpass import getpass  # To securely input password

# Import the main client class from aiograpi
from aiograpi import Client

# Import and apply nest_asyncio to allow running asyncio code easily in Jupyter
import nest_asyncio
nest_asyncio.apply()

print("Imports done, nest_asyncio applied.")

# --- 3. Client Instantiation ---
cl = Client()
print("Client object created.")

# --- 4. Authentication (Choose ONE method) ---

# IMPORTANT: Avoid hardcoding credentials directly in the notebook.
# Use environment variables or getpass for better security.

# Method A: Login with Username and Password (Use with caution!)
# -------------------------------------------------------------
username = os.environ.get("X23430133") or input("Enter Instagram Username: ")
password = os.environ.get("AnimalCrossing") or getpass("Enter Instagram Password: ")

async def login_with_password():
    print(f"Attempting login for user: {username}...")
    try:
        await cl.login(username, password)
        print("Login successful!")
        # Optional: Save session settings to avoid logging in next time
        settings_path = "session_settings.json"
        cl.dump_settings(settings_path)
        print(f"Session settings saved to {settings_path}")
        return True
    except Exception as e:
        print(f"Login failed: {e}")
        return False

# # Run the async login function
login_successful = asyncio.run(login_with_password())


# Method B: Login with Session ID (Generally Safer)
# -------------------------------------------------
# You need to get your 'sessionid' cookie value from your browser's
# developer tools after logging into instagram.com
# session_id = os.environ.get("INSTAGRAM_SESSIONID") or input("Paste your Instagram sessionid: ")

# async def login_with_sessionid():
#     print("Attempting login with sessionid...")
#     try:
#         cl.sessionid = session_id # Set the session ID directly
#         # You might need to trigger a request to validate the session,
#         # like fetching your own profile info
#         my_info = await cl.user_info_by_username(cl.username) # Or fetch your own ID if known
#         print(f"Session ID seems valid. Logged in as {my_info.username}")
#         # Optional: Save session settings
#         settings_path = "session_settings.json"
#         cl.dump_settings(settings_path)
#         print(f"Session settings saved to {settings_path}")
#         return True
#     except Exception as e:
#         print(f"Login via sessionid failed or session invalid: {e}")
#         return False

# # Run the async login function
# login_successful = asyncio.run(login_with_sessionid())

# Method C: Load Session from Saved Settings
# ------------------------------------------
# This is the preferred method after you've logged in successfully once
# using Method A or B and saved the settings.
settings_path = "session_settings.json"

async def load_session():
    if os.path.exists(settings_path):
        print(f"Attempting to load session from {settings_path}...")
        try:
            cl.load_settings(settings_path)
            # Verify the session is still valid by making a simple request
            # Example: Get self account info
            me = await cl.account_info()
            print(f"Session loaded successfully. Logged in as {me.username}")
            return True
        except Exception as e:
            print(f"Failed to load or validate session from file: {e}")
            print("You might need to login again using Method A or B.")
            return False
    else:
        print(f"Session file '{settings_path}' not found. Please login first using Method A or B.")
        return False

# Run the async load function
login_successful = asyncio.run(load_session())


In [ ]:


# --- 5. Basic API Call Example (Only run if login_successful is True) ---

async def get_user_data(username_to_lookup):
    if not login_successful:
        print("Cannot perform API call, login was not successful.")
        return None

    print(f"\nFetching info for user: {username_to_lookup}...")
    try:
        user_info = await cl.user_info_by_username(username_to_lookup)
        print("Successfully fetched user info!")
        # Print some details (user_info is an object, use .dict() or specific attributes)
        # Use .model_dump() for pydantic v2+ or .dict() for v1
        if hasattr(user_info, 'model_dump'):
             print(json.dumps(user_info.model_dump(), indent=2))
        elif hasattr(user_info, 'dict'):
             print(json.dumps(user_info.dict(), indent=2))
        else:
             print(user_info) # Fallback if methods don't exist

        # Example: Get user's first few medias
        # print(f"\nFetching media for user: {username_to_lookup}...")
        # medias = await cl.user_medias(user_info.pk, amount=5) # Fetch 5 posts
        # print(f"Fetched {len(medias)} medias.")
        # for media in medias:
        #     print(f" - Media ID: {media.pk}, Type: {media.media_type}, Caption: {media.caption_text[:50]}...")

        return user_info

    except Exception as e:
        print(f"An error occurred during API call: {e}")
        return None

# --- 6. Run the Example API Call ---
# Replace 'instagram' with the actual username you want to look up
target_username = 'instagram'

# Run the async function to get user data
# Because we used nest_asyncio, we can use await directly in the cell if login was successful
if login_successful:
    print("\nRunning API call example...")
    user_data = asyncio.run(get_user_data(target_username))
    if user_data:
        print("\nBoilerplate execution finished successfully.")
    else:
        print("\nBoilerplate execution finished with errors during API call.")
else:
    print("\nSkipping API call example because login failed or was skipped.")

# --- Optional: Logout (clears session) ---
# async def logout():
#     print("\nLogging out...")
#     await cl.logout()
#     print("Logged out.")
#     # Optionally remove settings file
#     if os.path.exists(settings_path):
#         os.remove(settings_path)
#         print(f"Removed {settings_path}")

# asyncio.run(logout())